# Decoding Quantum CSP Codes

In [ ]:
import json
import numpy as np
from tqdm import tqdm
import qecstruct as qc
import qecsim.paulitools as pt
import matplotlib.pyplot as plt

from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    apply_constraints,
    apply_bitflip_bias,
)
from examples.decoding.decoding import (
    decode_css,
    pauli_to_mps,
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
)

In [2]:
f = open(
    "/Users/aleksandrberezutskii/Desktop/data-csp-codes/batch_1/codes/qubits_30/code_0.json"
)
data = json.load(f)

In [4]:
data

{'x_stabs': [[1, 3, 5, 8, 10, 13, 14, 16, 27],
  [4, 8, 11, 12, 13, 14, 15, 18, 19, 21, 22, 24],
  [2, 3, 5, 8, 10, 15, 16, 17, 26, 29],
  [9, 11, 24, 25, 26, 27],
  [13, 16, 17, 18, 20, 22],
  [4, 5, 12, 20, 23, 28],
  [0, 5, 6, 8, 11, 17, 20, 21],
  [0, 1, 2, 6, 7, 9, 10, 11, 13, 14, 19, 24, 25],
  [6, 12, 14, 18, 19, 23],
  [2, 3, 4, 9, 16, 25, 28, 29],
  [6, 7, 9, 10, 12, 15, 28, 29],
  [1, 2, 6, 12, 13, 14, 16, 21, 22, 23, 27],
  [0, 2, 5, 7, 11, 26, 29]],
 'z_stabs': [[3, 5, 6, 8, 10, 11, 23, 25],
  [0, 6, 7, 9, 18, 22, 26, 29],
  [0, 2, 4, 10, 12, 14, 21],
  [4, 6, 10, 12, 13, 17, 19, 23, 28],
  [1, 9, 10, 11, 12, 13, 15, 16, 21, 23, 24, 26],
  [1, 5, 10, 16, 20, 29],
  [3, 6, 8, 12, 14, 19, 20, 21, 22, 25, 27],
  [0, 7, 9, 11, 15, 29],
  [1, 2, 3, 8, 21, 26, 27],
  [0, 1, 3, 5, 6, 14, 18, 20, 23, 28],
  [0, 2, 9, 10, 12, 13, 15, 17, 23, 24],
  [0, 8, 18, 19, 22, 26, 27],
  [2, 4, 7, 8, 16, 17, 28],
  [2, 5, 13, 20, 24, 25]],
 'num_qubits': 30}

In [3]:
data["x_stabs"]

[[1, 3, 5, 8, 10, 13, 14, 16, 27],
 [4, 8, 11, 12, 13, 14, 15, 18, 19, 21, 22, 24],
 [2, 3, 5, 8, 10, 15, 16, 17, 26, 29],
 [9, 11, 24, 25, 26, 27],
 [13, 16, 17, 18, 20, 22],
 [4, 5, 12, 20, 23, 28],
 [0, 5, 6, 8, 11, 17, 20, 21],
 [0, 1, 2, 6, 7, 9, 10, 11, 13, 14, 19, 24, 25],
 [6, 12, 14, 18, 19, 23],
 [2, 3, 4, 9, 16, 25, 28, 29],
 [6, 7, 9, 10, 12, 15, 28, 29],
 [1, 2, 6, 12, 13, 14, 16, 21, 22, 23, 27],
 [0, 2, 5, 7, 11, 26, 29]]

In [10]:
NUM_BITS = 60

In [11]:
state = create_simple_product_state(NUM_BITS, which="+")

In [12]:
code_constraint_sites = data["x_stabs"]
tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]
code_constraint_sites

[[1, 3, 5, 8, 10, 13, 14, 16, 27],
 [4, 8, 11, 12, 13, 14, 15, 18, 19, 21, 22, 24],
 [2, 3, 5, 8, 10, 15, 16, 17, 26, 29],
 [9, 11, 24, 25, 26, 27],
 [13, 16, 17, 18, 20, 22],
 [4, 5, 12, 20, 23, 28],
 [0, 5, 6, 8, 11, 17, 20, 21],
 [0, 1, 2, 6, 7, 9, 10, 11, 13, 14, 19, 24, 25],
 [6, 12, 14, 18, 19, 23],
 [2, 3, 4, 9, 16, 25, 28, 29],
 [6, 7, 9, 10, 12, 15, 28, 29],
 [1, 2, 6, 12, 13, 14, 16, 21, 22, 23, 27],
 [0, 2, 5, 7, 11, 26, 29]]